# CF cơ bản - Memmory based - item base CF

In [12]:
import pandas as pd
import numpy as np

TRAIN_PATH = "data/train.txt"
TEST_PATH  = "data/test.txt"

# Đọc dữ liệu
df_train = pd.read_csv(
    TRAIN_PATH,
    sep=r"\s+",
    header=None,
    names=["userid","movieid","rating"],
    engine="python",
)
df_test = pd.read_csv(
    TEST_PATH,
    sep=r"\s+",
    header=None,
    names=["userid","movieid"],
    engine="python",
)

# Thay .append(...) bằng pd.concat([...])
all_u = pd.concat([df_train["userid"], df_test["userid"]], ignore_index=True)
all_i = pd.concat([df_train["movieid"], df_test["movieid"]], ignore_index=True)

# Tạo mapping 0..n-1 (giữ thứ tự xuất hiện)
uid_uniques = all_u.drop_duplicates()
iid_uniques = all_i.drop_duplicates()
uid_map = pd.Series(np.arange(len(uid_uniques), dtype=np.int32), index=uid_uniques.values)
iid_map = pd.Series(np.arange(len(iid_uniques), dtype=np.int32), index=iid_uniques.values)

# Ánh xạ sang index liên tục
df_train["u_idx"] = df_train["userid"].map(uid_map).astype(np.int32)
df_train["i_idx"] = df_train["movieid"].map(iid_map).astype(np.int32)
df_test["u_idx"]  = df_test["userid"].map(uid_map).astype(np.int32)
df_test["i_idx"]  = df_test["movieid"].map(iid_map).astype(np.int32)

# Số lượng user/item chuẩn nên lấy theo kích thước mapping (bao gồm cả test)
n_users = uid_map.size
n_items = iid_map.size

print(f"n_users={n_users}, n_items={n_items}, train_rows={len(df_train)}, test_rows={len(df_test)}")
print(df_train.head())


n_users=943, n_items=1682, train_rows=90570, test_rows=9430
   userid  movieid  rating  u_idx  i_idx
0       1        1       5      0      0
1       1        2       3      0      1
2       1        3       4      0      2
3       1        4       3      0      3
4       1        5       3      0      4


In [13]:
from scipy import sparse

# Ma trận R dạng CSR: U x I
R = sparse.coo_matrix(
    (df_train["rating"].astype(np.float32), (df_train["u_idx"], df_train["i_idx"])),
    shape=(n_users, n_items),
).tocsr()

# Baseline: global mean + user bias + item bias (ridge nhỏ để ổn định)
mu = df_train["rating"].mean()

# Tính b_u, b_i theo cách lặp coordinate descent đơn giản
lambda_reg = 10.0  # có thể tinh chỉnh
b_u = np.zeros(n_users, dtype=np.float32)
b_i = np.zeros(n_items, dtype=np.float32)

# Chuẩn bị chỉ mục thuận tiện
users_nonzero_items = [R[u].indices for u in range(n_users)]
items_nonzero_users = [R[:, i].indices for i in range(n_items)]

for _ in range(3):  # 2-3 vòng là đủ cho baseline
    # update b_u
    for u in range(n_users):
        idx_i = users_nonzero_items[u]
        if len(idx_i) == 0:
            b_u[u] = 0.0
            continue
        r_ui = R[u, idx_i].toarray().ravel()
        numer = (r_ui - mu - b_i[idx_i]).sum()
        b_u[u] = numer / (lambda_reg + len(idx_i))
    # update b_i
    for i in range(n_items):
        idx_u = items_nonzero_users[i]
        if len(idx_u) == 0:
            b_i[i] = 0.0
            continue
        r_ui = R[idx_u, i].toarray().ravel()
        numer = (r_ui - mu - b_u[idx_u]).sum()
        b_i[i] = numer / (lambda_reg + len(idx_u))

def predict_baseline(u, i, clip=(1,5)):
    val = mu + b_u[u] + b_i[i]
    if clip is not None:
        val = np.clip(val, clip[0], clip[1])
    return float(val)


In [14]:
from sklearn.neighbors import NearestNeighbors

# Tính ma trận “residual” theo baseline: e_ui = r_ui - (mu + b_u[u] + b_i[i])
# Làm ở dạng sparse CSR: eR có cùng cấu trúc nonzero với R
data = []
rows = []
cols = []
for u in range(n_users):
    idx_i = users_nonzero_items[u]
    if len(idx_i) == 0: 
        continue
    r_ui = R[u, idx_i].toarray().ravel().astype(np.float32)
    resid = r_ui - (mu + b_u[u] + b_i[idx_i])
    nz = np.where(np.abs(resid) > 1e-6)[0]
    if len(nz):
        rows.extend([u]*len(nz))
        cols.extend(idx_i[nz])
        data.extend(resid[nz])
eR = sparse.csr_matrix((data, (rows, cols)), shape=(n_users, n_items))

# Dùng transpose (items x users) để tìm KNN theo item
X = eR.T.tocsr()  # shape: (n_items, n_users)

K = 50        # số láng giềng mỗi item
metric = "cosine"
nbrs = NearestNeighbors(n_neighbors=min(K+1, n_items), metric=metric, algorithm="auto")
nbrs.fit(X)

# Lấy top-K hàng xóm cho toàn bộ item (precompute để dự đoán nhanh)
# kneighbors trả về cả chính nó ở vị trí 0 => ta bỏ đi vị trí 0
distances, indices = nbrs.kneighbors(X, return_distance=True)
# cosine distance d in [0,2]; sim = 1 - d (xấp xỉ khi vector đã chuẩn)
sims_full = 1.0 - distances
neighbors = indices[:, 1:K+1]
sims      = sims_full[:, 1:K+1]

# Hàm dự đoán CF: trọng số theo similarity + shrinkage
def predict_item_cf(u, i, shrink=10.0, clip=(1,5)):
    # láng giềng của item i
    neigh_i = neighbors[i]
    sim_i   = sims[i]
    # Lọc chỉ giữ các item mà user u đã có residual (đã xem)
    user_items = set(users_nonzero_items[u])
    mask = np.array([j in user_items for j in neigh_i])
    if not mask.any():
        # fallback: baseline
        return predict_baseline(u, i, clip)
    neigh_i = neigh_i[mask]
    sim_i   = sim_i[mask]

    # residual của user u trên các item láng giềng
    # Lấy trực tiếp từ eR[u, neigh_i]
    if len(neigh_i) == 0:
        return predict_baseline(u, i, clip)

    # số user overlap của từng láng giềng với item i để shrink (ước lượng nhanh)
    # Ở đây dùng độ dài giao người dùng giữa 2 item ~ qua dot sản phẩm nhị phân
    # Đơn giản hơn: dùng số người dùng đã rating neighbor (đã đủ để shrink thô)
    support = np.array([len(items_nonzero_users[j]) for j in neigh_i], dtype=np.float32)

    # shrinkage: s' = s * (support / (support + shrink))
    weights = sim_i * (support / (support + shrink))

    # residuals của user u ở các item hàng xóm:
    # eR[u, neigh_i] lấy dạng dense mảng 1d
    e_vals = eR[u, neigh_i].toarray().ravel()
    # Nếu tất cả 0 (có thể do tròn số), fallback baseline
    if np.all(np.abs(e_vals) < 1e-12):
        return predict_baseline(u, i, clip)

    # Dự đoán: baseline + weighted sum residuals
    denom = np.abs(weights).sum()
    if denom < 1e-8:
        return predict_baseline(u, i, clip)
    cf_part = (weights * e_vals).sum() / denom
    val = mu + b_u[u] + b_i[i] + cf_part
    if clip is not None:
        val = np.clip(val, clip[0], clip[1])
    return float(val)


In [15]:
# ===== 4) Dự đoán & Xuất submission theo định dạng mẫu =====
# Giả sử bạn đã có các biến/hàm từ phần CF:
# - df_test với cột u_idx, i_idx (giữ nguyên thứ tự hàng gốc)
# - predict_item_cf(u, i, shrink=..., clip=(1,5))  # có fallback baseline

CLIP = (1, 5)  # đổi nếu thang điểm khác
SHRINK = 10.0  # siêu tham số CF của bạn

# Tạo mảng dự đoán theo đúng thứ tự hàng df_test
preds = np.empty(len(df_test), dtype=np.float32)
for idx, (u, i) in enumerate(zip(df_test["u_idx"].values, df_test["i_idx"].values)):
    preds[idx] = predict_item_cf(u, i, shrink=SHRINK, clip=CLIP)

# Lập DataFrame submission: Id bắt đầu từ 1, cột Score là dự đoán
submission = pd.DataFrame({
    "Id": np.arange(1, len(df_test) + 1, dtype=np.int64),
    "Score": preds
})

# Ghi ra CSV đúng format (không có index, có thể cố định 6 chữ số thập phân)
SUB_PATH = "output/CFsubmission.csv"
submission.to_csv(SUB_PATH, index=False, float_format="%.6f")
print("Saved:", SUB_PATH, submission.shape)
print(submission.head())


Saved: output/CFsubmission.csv (9430, 2)
   Id  Score
0   1    1.0
1   2    1.0
2   3    1.0
3   4    1.0
4   5    1.0
